This notebook explores the relationship between rate distortion and privacy.

First, some preliminaries:

In [1]:
:set -XRebindableSyntax
:set -XNoImplicitPrelude

import Privacy.Prelude

We'll need some random variables:

We can build a huffman tree from these variables:

In [25]:
import Algebra.Information.Huffman
import Algebra.Information.Tree
import qualified Data.List.NonEmpty as NonEmpty
import Data.Functor.Contravariant
import Data.Bifunctor
import Data.Monoid
import qualified Data.Map.Strict as Map
import Algebra.Information.Huffman.Alphabetic
import Control.Monad
import Text.PrettyPrint hiding ((<>))

-- agesData = [14,16,21,17,15,25,26,33,27,32,55,31,52,65,48,74,63,96,93,108,87,129,139,149,144,195,178,186,208,232,208,210,218,223,250,265,241,275,269,278,259,249,272,236,271,259,249,234,246,232,217,186,167,172,174,175,146,128,110,122,113,91,84,77,70,70,49,47,44,38,35,33,25,23,15,18,9,12,11,6,5,4,6,5,1,1,0,1,2,6]
  
-- agesData

-- chunked = [226,685,1647,2437,2507,1597,683,187,31]

--chunked

tree
    = first getSum
    . fmap fst
    . huffmanTree defaultComparison (Sum . snd) 
    . NonEmpty.fromList

:opt svg

-- agesTree = tree (zip [0..] agesData)
-- --agesTree
-- alphAgesTree = alphHuffman (Map.fromList (zip [0..] agesData))
-- --alphAgesTree

smalls = [1,2,1,3,2,0,1,0,3]

--tree (map (flip (,) 1) smalls)

import Data.List.NonEmpty (NonEmpty(..))
import Data.Semigroup.Foldable (toNonEmpty, foldMap1)

treeSmalls xs fullTree = (\y -> rng $ fromRight (followPath (mapping Map.! y) truncd))
  where
    mapping = codeBook fullTree
    truncd = truncateTree (>=2) (const (:|[])) fullTree
    fromRight (Right x) = x
    fromRight (Left xs) = foldMap1 toNonEmpty xs
    rng xs = (minimum xs, maximum xs)

huffTreeSmalls xs = treeSmalls xs (tree (map (flip (,) 1) xs))
alphTreeSmalls xs = treeSmalls xs (alphHuffman (Map.fromListWith (+) $ map (flip (,) 1) xs))

import Data.List (inits)

anons anonfn xs = map f (inits xs)
  where
    f ys = map fn ys
      where
        fn = anonfn ys
   
import Text.PrettyPrint hiding ((<>))

prettyAnon xs = vcat $ zipWith f [1..] (tail xs)
  where
    f i ys = int i <+> char '&' <+> hsep (punctuate (text " &") (map g ys)) <+> (char '\\' <> char '\\')
    g (x,y) | x == y = int x <> text "    "
            | otherwise = lbrack <> int x <> comma <> int y <> rparen
   
--prettyAnon (anons huffTreeSmalls smalls)
prettyAnon (anons alphTreeSmalls smalls)

-- ptr
--     = prettyTree
--       int
-- --       (\x -> braces $ char '$' <> lbrack <> int x <> comma <> int (x+10) <> rparen <> char '$')
      
-- -- ptr agesTree
-- -- ptr alphAgesTree
-- agesTree
-- alphAgesTree
-- agesTree

1 & 1     \\
2 & [1,2) & [1,2) \\
3 & [1,2) & [1,2) & [1,2) \\
4 & 1     & [2,3) & 1     & [2,3) \\
5 & 1     & [2,3) & 1     & [2,3) & [2,3) \\
6 & [0,1) & [2,3) & [0,1) & [2,3) & [2,3) & [0,1) \\
7 & [0,1) & [2,3) & [0,1) & [2,3) & [2,3) & [0,1) & [0,1) \\
8 & 1     & [2,3) & 1     & [2,3) & [2,3) & 0     & 1     & 0     \\
9 & 1     & 2     & 1     & 3     & 2     & 0     & 1     & 0     & 3     \\

In [5]:
-- alphAgesTree
-- optAlphHuffman ages
import Prelude
import Control.Lens
import Data.List (groupBy)
import Algebra.Information.Histogram
import Data.Function (on)
import Control.Arrow hiding (first)
import Control.Monad
import Data.Ratio

kdistortion k = histogramDistortion . fmap Histogram . truncateTree (>=k) (flip Map.singleton)

bookend [] = []
bookend [x] = [x]
bookend [x,y] = [x,y]
bookend xs = [head xs, last xs]

kratedistortion :: Tree Integer Rational -> [(Double,Double)]
kratedistortion tr
    = map (((`kdistortion` (first fromInteger tr)) . fromInteger) &&& (% measure tr))
  >>> groupBy ((==) `on` fst)
  >=> bookend
  >>> map (fromRational *** fromRational)
  $ [2 .. measure tr]

-- rages <- randAges

-- ragesi = Histogram $ Map.fromListWith (+) . map (flip (,) (1 :: Integer)) $ rages

-- avg = average ragesi

-- import Control.Lens

import Prelude (Num)
import Prelude
mapHistNum :: (Num n, Ord b) => (a -> b) -> Histogram n a -> Histogram n b
mapHistNum = over (histIso . mapping (_Unwrapping Sum) . from histIso) . mapHist

-- mse 3 5

-- sqrt (fromRational (average (mapHistNum (mse avg) ragesi))) :: Double

mapM_ (\(x,y) -> putStr (show x) *> putStr " " *> print y) (kratedistortion agesTree)
--mapM_ (\(x,y) -> putStr (show x) *> putStr " " *> print y) (kratedistortion (first toEnum alphAgesTree))

--histogramDistortion (Histogram (Map.fromList [(1,1)]))

7.0e-4 2.0e-4
1.65e-3 3.0e-4
1.65e-3 4.0e-4
8.775e-3 5.0e-4
1.1275e-2 6.0e-4
5.7887987012987016e-2 7.0e-4
5.7887987012987016e-2 9.0e-4
6.034632034632035e-2 1.0e-3
6.034632034632035e-2 1.1e-3
8.058268398268398e-2 1.2e-3
4.064870683982684 1.3e-3
4.064870683982684 1.5e-3
4.086612555903867 1.6e-3
4.086612555903867 1.7e-3
8.752212555903867 1.8e-3
8.752212555903867 1.9e-3
15.411994607185918 2.0e-3
15.411994607185918 2.3e-3
15.488526526377838 2.4e-3
15.488526526377838 2.5e-3
21.214291832500287 2.6e-3
22.29268618544146 2.7e-3
29.019915742091708 2.8e-3
29.019915742091708 3.1e-3
29.134920660124497 3.2e-3
29.141520660124495 3.3e-3
30.729460958631957 3.4e-3
30.729460958631957 3.5e-3
30.73130818085418 3.6e-3
30.73130818085418 4.0e-3
32.266953037202654 4.1e-3
32.266953037202654 4.6e-3
32.50359009034275 4.7e-3
32.50359009034275 4.8e-3
39.12839009034275 4.9e-3
39.12839009034275 5.0e-3
39.830529559991994 5.1e-3
39.830529559991994 5.2e-3
39.84132201282218 5.3e-3
39.84132201282218 5.7e-3
43.9156599519322

For $k$-anonymity, we cut off revealing the tree at a certain point. Below that point, we simply return a summary of the subtree.

We can see what value of $k$ is provided after cutting off at a certain level by taking the minimum size of subtrees at that level:

In [5]:
-- randVar :: IO Rational
-- randVar = (toEnum.fromEnum) <$>
--     withSystemRandom (genContVar (normalDistr 40 10) `asTypeOf`
--                       const (undefined :: IO Double))
                      
-- randVars <- replicateM 100 randVar

-- mapM_ (print.fromEnum) randVars


-- --mapM_ print (kratedistortion (tree randVars))
-- --mapM_ print (kratedistortion (alphHuffman (Map.fromListWith (+) . map (flip (,) 1) $ randVars)))

15
59
25
39
51
46
37
30
31
49
23
31
57
26
42
37
34
29
39
40
23
40
45
46
36
36
33
34
37
52
48
36
44
50
40
61
39
45
13
58
60
54
47
48
34
41
44
44
49
51
43
30
36
53
43
27
55
43
34
24
61
38
31
54
47
32
20
46
41
34
47
32
37
39
38
42
58
45
46
30
48
39
43
33
30
71
28
40
58
49
45
37
23
47
40
38
27
43
34
32

We can plot that $k$ against the number of bits revealed

In [6]:
import Graphics.Rendering.Chart.Plot.Lines
import Graphics.Rendering.Chart.Plot
import Graphics.Rendering.Chart.Layout
import Graphics.Rendering.Chart (toRenderable)
import Control.Lens ((.~))
import Data.Default


plotRateDistortion xs
    = toRenderable
    $ layout_plots .~ [ toPlot
                      $ PlotLines
                          "random vars"
                          defaultPlotLineStyle
                          [zip xs [0..]]
                          []
                      ]
    $ layout_x_axis.laxis_title .~ "k"
    $ layout_y_axis.laxis_title .~ "Level"
    $ def

plotRateDistortion (rates tree)

<interactive>:1:21: error:
    • Variable not in scope: rates :: ([()] -> Tree Integer ()) -> [x0]
    • Perhaps you meant ‘rages’ (line 1)

In [7]:
:ext FlexibleContexts

import           Control.Monad.State
import qualified Data.Map.Strict     as Map
import           Data.Foldable       (Foldable(..))
import           Algebra.Information.Huffman.Alphabetic (optAlphTree)
import           Data.Bifunctor (first)

buildAlphHuffman :: Histogram Count a -> Tree Count a
buildAlphHuffman xs = evalState (go c) (Map.toList (getHistogram xs))
  where
    c = fromEnum (sum (getCount <#$> getHistogram xs))
    go n = do
      (x,Count i) <- gets head
      if fromEnum i >= n
        then do
          modify tail
          pure (Leaf (Count i) x)
        else do
          let m = n `div` 2
          ls <- go m
          let nxt = n - fromEnum (getCount (measure ls))
          if nxt <= 0 then pure ls else do
            rs <- go nxt
            pure (Node (measure ls <> measure rs) ls rs)
            
buildOptAlphHuffman :: Ord a => Histogram Count a -> Tree Count a
buildOptAlphHuffman xs
    = first (Count . toEnum) 
    $ optAlphTree
    [ (fromEnum i, x)
    | (x,Count i) <- Map.toList (getHistogram xs) ]

alphHuffmanTree :: (Foldable1 f, Ord a) => f a -> Tree Count a
alphHuffmanTree = buildAlphHuffman . generalizeTo (histogramOf count)

<interactive>:1:58: error: Module ‘Algebra.Information.Huffman.Alphabetic’ does not export ‘optAlphTree’

In [9]:
variance xs = avg (map (mse m) xs)
  where 
    m = avg xs

avg xs = sum xs / fromIntegral (length xs)

mse x y = let z = x - y in z * z

stddev = sqrt . variance

